In [91]:
import time  
import math
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import copy

In [92]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

In [93]:
print(f'Using:{device}')

Using:cpu


### HyperParameters

In [94]:
epochs=14
batch_size=100
learning_rate=0.001
learning_momentum=0.9
in_channels=3
num_classes=10
model_save_location=r'D:\DeepLearning\CIFAR10_DenseNet161'

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Creating a transform

In [95]:
data_transforms ={'train':transforms.Compose([
                            transforms.ToTensor(),
                            transforms.RandomHorizontalFlip(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                 'val':transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                 } 

#### Loading CIFAR-10 Dataset

In [96]:
#Images in this dataset have 3 channels and are 32x32

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transforms['train'],
                                download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=data_transforms['val'],
                                download=True)

Files already downloaded and verified
Files already downloaded and verified


#### Make DataLoader

In [97]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

train_size = len(train_dataset)
test_size = len(test_dataset)

### Showing some images

In [98]:
#To-Do

### Training

In [99]:
def train(model, train_loader, loss_func, optimizer, epochs, scheduler):
    since = time.time()
    model.train()
    for epoch in range(epochs):
        print(f'Epoch:{epoch+1}/{epochs}')
        batch_loss = 0
        running_loss = 0
        running_corrects = 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            output = model(images)
            _, preds = torch.max(output, 1)
            
            loss = loss_func(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch_loss += loss.item() * images.size(0)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds==labels.data)
            
            if i+1 % batch_size == 0:
                time_elapsed = time.time() - since
                batch_loss /= batch_size
                print(f'Epoch:{epoch+1}/{epochs}, Batch Loss:{batch_loss}, Time Elapsed:{time_elapsed // 60} minutes:{time_elapsed % 60} seconds')
                batch_loss = 0      
        # Do step in learning rate after each epoch   
        scheduler.step()  

        epoch_loss = running_loss / train_size
        epoch_acc = 100 * running_corrects.double() / train_size

        time_epoch = time.time() - since                  
        print(f'Epoch:{epoch}/{epochs}, Epoch Loss:{epoch_loss}, Epoch Accuracy:{epoch_acc}')
    # Training is over
    time_training = time.time()-since
    print(f'Finish Training in:{time_training // 60} minutes:{time_training % 60} seconds')
    try:
        torch.save(model.state_dict(), model_save_location)
    except:
        print("Something wrong happened when saving the model. Check if you saving location exists.")

### Evaluating

In [100]:
def eval(model, eval_dataset):
    since = time.time()
    
    total_images = 0
    total_correct = 0
    
    model.eval()
    with torch.no_grad():
        for images, labels in eval_dataset:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predictions = torch.max(outputs.data, 1)
            
            total_images += images.size(0)
            total_correct += torch.sum(predictions==labels.data)
    #Finished Evaluating
    time_spent = time.time() - since
    percent_acc = 100 * total_correct / total_images
    print(f'Finished Evaluating in:{time_spent // 60} minutes:{time_spent % 60} seconds')
    print('Model Prediction Accuracy:{:.2f}%'.format(percent_acc))

### Using Transfer Learning

In [101]:
## Loading in DenseNet161

model = models.densenet161(pretrained=True)

# Going to freeze the base of the dense net
 #for param in model.parameters():
    # param.requires_grad = False

num_fltrs = model.classifier.in_features
# Make the last layer predict probabilities for 10 classes
model.classifier = nn.Linear(num_fltrs, 10)
#Transfer model to gpu if using it
model.to(device)

#CrossEntropyLoss includes softmax
loss_func = nn.CrossEntropyLoss()

#Optimizer
optimizer_func = optim.SGD(model.parameters(), lr=learning_rate, momentum=learning_momentum) 

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_func, step_size=7, gamma=0.1)

### Actually training

In [102]:
train(model=model,train_loader=train_loader, loss_func=loss_func, optimizer=optimizer_func, epochs=epochs, scheduler=exp_lr_scheduler)

Epoch:1/14


KeyboardInterrupt: 

### Evaluating

In [ ]:
eval(model=model, eval_dataset=test_loader)